In [ ]:
import numpy as np
import math
from scipy import linalg as LA
import matplotlib.pyplot as plt
from scipy.misc import derivative

# function to take partial derivative
##############################################
def partial_derivative(function, variable=0, point=[]):
    arguments = point[:]
    def wraps(x):
        arguments[variable] = x
        return function(*arguments)
    return derivative(wraps, point[variable], dx=np.sqrt(np.finfo(float).eps))

In [ ]:
##############################################
###                 system                 ###
##############################################

b, d, q = 2, 3, 3  # decimation parameter, dimension, state number q

# logarithms of R1(aa), R2(ab), R3(bc), R4(bb)
##############################################
def lnR1(ja, jb, h):
    return 2*ja+4*h+math.log(1+(q-1)*math.exp(-2*ja-2*h))
def lnR2(ja, jb, h):
    return ja+3*h+math.log(1+math.exp(jb-ja-2*h)+(q-2)*math.exp(-ja-2*h))
def lnR3(ja, jb, h):
    return jb+math.log(2+math.exp(2*h-jb)+(q-3)*math.exp(-jb))
def lnR4(ja, jb, h):
    return 2*jb+math.log(1+math.exp(2*h-2*jb)+(q-2)*math.exp(-2*jb))

# recursion relations for Ja', Jb', H', and G'(G=0)
# (with Migdal-Kadanoff bond moving)
##############################################
def Ja(ja, jb, h):
    jja, jjb, hh = b**(d-1)*ja, b**(d-1)*jb, b**(d-1)*h
    return lnR1(jja,jjb,hh)+lnR3(jja,jjb,hh)-2*lnR2(jja,jjb,hh)
def Jb(ja, jb, h):
    jja, jjb, hh = b**(d-1)*ja, b**(d-1)*jb, b**(d-1)*h
    return lnR4(jja,jjb,hh)-lnR3(jja,jjb,hh)
def H(ja, jb, h):
    jja, jjb, hh = b**(d-1)*ja, b**(d-1)*jb, b**(d-1)*h
    return lnR2(jja,jjb,hh)-lnR3(jja,jjb,hh)
def G(ja, jb, h):
    jja, jjb, hh = b**(d-1)*ja, b**(d-1)*jb, b**(d-1)*h
    return lnR3(jja,jjb,hh)

# T matrix
##############################################
def T_matrix(ja, jb, h):
    
    X = partial_derivative(G,  0, [ja,jb,h])  # dG' /dJa 
    Y = partial_derivative(Ja, 0, [ja,jb,h])  # dJa'/dJa
    Z = partial_derivative(Jb, 0, [ja,jb,h])  # dJb'/dJa
    Q = partial_derivative(H,  0, [ja,jb,h])  # dH' /dJa
    
    K = partial_derivative(G,  1, [ja,jb,h])   # dG' /dJb
    L = partial_derivative(Ja, 1, [ja,jb,h])   # dJa'/dJb
    M = partial_derivative(Jb, 1, [ja,jb,h])   # dJb'/dJb
    N = partial_derivative(H,  1, [ja,jb,h])   # dH' /dJb
    
    O = partial_derivative(G,  2, [ja,jb,h])   # dG' /dH
    P = partial_derivative(Ja, 2, [ja,jb,h])   # dJa'/dH
    R = partial_derivative(Jb, 2, [ja,jb,h])   # dJb'/dH
    S = partial_derivative(H,  2, [ja,jb,h])   # dH' /dH
    
    return [[b**d, X,   K,   O],
            [0,    Y,   L,   P],
            [0,    Z,   M,   R],
            [0,    Q,   N,   S]]

In [ ]:
##############################################
###             just RG flows              ###
##############################################

#Jc = 0.1649552287195008
j_initial = 0.16
h_initial = 0
n = 10

#q = 3
ja, jb, h = j_initial, j_initial, h_initial
print("Ja                      Jb                      H\n")
for i in range(n):
    print(ja, "   ", jb, "   ", h)
    ja, jb, h = Ja(ja,jb,h), Jb(ja,jb,h), H(ja,jb,h)
ja, jb, h = j_initial, j_initial, h_initial
print("\n\nln[R(aa)]             ln[R(bb)]             ln[R(ab)]            ln[R(bc)]\n")
for i in range(n):
    print(lnR1(ja, jb, h), "  ", lnR4(ja, jb, h), "  ", lnR2(ja, jb, h), "  ", lnR3(ja, jb, h))
    ja, jb, h = Ja(ja,jb,h), Jb(ja,jb,h), H(ja,jb,h)